In [ ]:
import React, { useState, useEffect, useCallback, useRef } from 'react';
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  shorthands,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";
import SubgraphResults from './SubgraphResults';

const useStyles = makeStyles({
  // ... (keep all existing styles)

  chatContent: {
    display: "flex",
    flexDirection: "column",
    height: "calc(100% - 60px)", // Adjust for input area height
    overflow: "hidden",
  },
  subgraphContainer: {
    marginBottom: "16px",
    overflowY: "auto",
    maxHeight: "30%", // Adjust as needed
  },
  messagesContainer: {
    flexGrow: 1,
    overflowY: "auto",
    display: "flex",
    flexDirection: "column",
    gap: "16px",
    marginBottom: "16px",
  },
});

const DynamicProgressBar = ({ currentStep }) => {
  // ... (keep existing implementation)
};

const EmailGenerator = () => {
  const styles = useStyles();
  // ... (keep all existing state variables)
  const [subgraphResults, setSubgraphResults] = useState(null);

  // ... (keep all existing functions like fetchUserId, fetchUserConfig, etc.)

  const handleUpdate = useCallback((data) => {
    console.log("handleUpdate received data:", data);
    if (data.type === 'step') {
      setCurrentStep(data.content);
      console.log("Updated currentStep:", data.content);
    } else if (data.type === 'final') {
      finalResponseRef.current = data.content;
      console.log("Received final response:", data.content);
    } else if (data.type === 'subgraph_results') {
      setSubgraphResults(data.content);
      console.log("Received subgraph results:", data.content);
    }
  }, []);

  const handleAction = useCallback(async (userInput) => {
    // ... (keep existing implementation)
    // Make sure to call handleUpdate with subgraph results when received from the API
  }, [userConfig, getEmailContent]);

  // ... (keep other existing functions)

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatContent}>
          {subgraphResults && (
            <div className={styles.subgraphContainer}>
              <SubgraphResults results={subgraphResults} />
            </div>
          )}
          <div className={styles.messagesContainer}>
            {chatMessages.map((message, index) => (
              <div
                key={index}
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
            ))}
          </div>
        </div>
        {isProcessing && (
          <>
            <DynamicProgressBar currentStep={currentStep} />
            <div>Debug - Current Step: {currentStep}</div>
          </>
        )}
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
            disabled={isProcessing}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  // ... (keep the rest of the component, including the return statement for the initial view)
};

export default EmailGenerator;